In [1]:
pip install pandas numpy scikit-learn joblib


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image

# Data generators (unchanged)
train_dir = r"C:\Users\VINIL\Desktop\soil_type_pred_ML\notebooks\Soil Classification.v4i.folder\train"
test_dir = r"C:\Users\VINIL\Desktop\soil_type_pred_ML\notebooks\Soil Classification.v4i.folder\test"
datagen = ImageDataGenerator(rescale=1./255,rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.2)
train_generator = datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=32,class_mode='categorical',subset="training")
val_generator = datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=32,class_mode='categorical',subset="validation")
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,target_size=(224,224),batch_size=32,class_mode='categorical')

# Optimized model for smaller .keras file
model = Sequential([
    Conv2D(16,(3,3),activation='relu',input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64,activation='relu'),
    Dense(5,activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
history = model.fit(train_generator,validation_data=val_generator,epochs=10,verbose=1)
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# This will save a smaller .keras file
# model.save("soil_classification_model.keras")

def predict_image(img_path):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array,axis=0)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    print(f"Predicted Class: {predicted_class}")

Found 1880 images belonging to 5 classes.
Found 467 images belonging to 5 classes.
Found 80 images belonging to 5 classes.


c:\Users\VINIL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\VINIL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 20s 331ms/step - accuracy: 0.5977 - loss: 1.1363 - val_accuracy: 0.7752 - val_loss: 1.1169
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 19s 317ms/step - accuracy: 0.8731 - loss: 0.3730 - val_accuracy: 0.8073 - val_loss: 0.7004
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 19s 323ms/step - accuracy: 0.8898 - loss: 0.3157 - val_accuracy: 0.7923 - val_loss: 1.0392
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 19s 321ms/step - accuracy: 0.9074 - loss: 0.2898 - val_accuracy: 0.7923 - val_loss: 0.8212
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 19s 324ms/step - accuracy: 0.9203 - loss: 0.2397 - val_accuracy: 0.7816 - val_loss: 1.0478
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 19s 325ms/step - accuracy: 0.9192 - loss: 0.2219 - val_accuracy: 0.8030 - val_loss: 1.1206
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 19s 324ms/step - accuracy: 0.9174 - loss: 0.2184 - val_accuracy: 0.7923 - val_loss: 1.0184
Epoch 8/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 19s 325ms/step - accuracy: 0.9156 - loss: 0.2151 - val_accu

In [3]:
import os
import pickle
import tensorflow as tf
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# 🔹 Paths Configuration
models_folder = r"C:\Users\VINIL\Desktop\soil_type_pred_ML\flask_app\models"
os.makedirs(models_folder, exist_ok=True)

# Define file paths
cnn_model_path = os.path.join(models_folder, "soil_cnn_model.keras")  # .keras instead of .h5
class_indices_path = os.path.join(models_folder, "class_indices.pkl")

# 🔹 Save the Trained CNN Model in Keras Format
# Assuming you have a trained model already (named `model`)
model.save(cnn_model_path)
logging.info(f"CNN model saved successfully at: {cnn_model_path}")

# 🔹 Save class indices (from your train_generator)
# Assuming you have already loaded train_generator and it has class_indices
with open(class_indices_path, "wb") as ci_file:
    pickle.dump(train_generator.class_indices, ci_file)
logging.info(f"Class indices saved successfully at: {class_indices_path}")

print("CNN model and class indices saved successfully in:", models_folder)


INFO:root:CNN model saved successfully at: C:\Users\VINIL\Desktop\soil_type_pred_ML\flask_app\models\soil_cnn_model.keras
INFO:root:Class indices saved successfully at: C:\Users\VINIL\Desktop\soil_type_pred_ML\flask_app\models\class_indices.pkl


CNN model and class indices saved successfully in: C:\Users\VINIL\Desktop\soil_type_pred_ML\flask_app\models
